## Uso de Scikit-learn para Construir Pipelines de Preprocesamiento 🛠️🔗

### Introducción

En las clases anteriores, vimos cómo aplicar individualmente diversas técnicas de limpieza y transformación de datos. Si bien esto es útil para entender cada paso, en la práctica, aplicar múltiples transformaciones secuencialmente puede volverse tedioso, propenso a errores y difícil de gestionar, especialmente cuando se trabaja con conjuntos de datos de entrenamiento y prueba.

Scikit-learn ofrece una herramienta poderosa para encadenar múltiples pasos de preprocesamiento (y modelado): los **Pipelines**. Un `Pipeline` permite ensamblar varias transformaciones y, opcionalmente, un estimador final (como un modelo de clasificación o regresión) en un único objeto.

Además, para aplicar diferentes transformaciones a diferentes columnas de nuestro dataset (por ejemplo, escalar columnas numéricas y codificar columnas categóricas), usaremos `ColumnTransformer`.

**Ventajas de usar Pipelines y ColumnTransformer:**
* **Simplicidad y Claridad:** El código se vuelve más organizado y legible.
* **Consistencia:** Asegura que los mismos pasos de preprocesamiento se apliquen de manera idéntica a diferentes conjuntos de datos (ej. entrenamiento y prueba).
* **Prevención de Fuga de Datos (Data Leakage):** Ayuda a aplicar correctamente `fit` solo en los datos de entrenamiento y `transform` tanto en entrenamiento como en prueba.
* **Facilidad de Experimentación:** Permite cambiar o ajustar pasos del preprocesamiento fácilmente.
* **Preparación para Despliegue:** Un pipeline completo puede ser guardado y cargado para usarse en producción.

### 1. Configuración e Importación de Librerías

In [1]:
import pandas as pd
import numpy as np

# Para Pipelines y ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Para Imputación
from sklearn.impute import SimpleImputer

# Para Escalado y Codificación
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# Para dividir el dataset (aunque en este notebook nos enfocaremos en el preprocesador)
from sklearn.model_selection import train_test_split

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

### 2. Carga del Dataset (Titanic)

Cargaremos la versión "cruda" del dataset del Titanic para demostrar cómo un pipeline puede manejar varias tareas de preprocesamiento a la vez. En un flujo de trabajo de Machine Learning real, dividiríamos los datos en conjuntos de entrenamiento y prueba ANTES de ajustar cualquier transformador para evitar la fuga de datos. Para esta demostración del pipeline de preprocesamiento, nos centraremos en construir el preprocesador, pero es crucial recordar este orden.

In [2]:
url_titanic = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'
try:
    df_titanic_raw = pd.read_csv(url_titanic)
    print("Dataset del Titanic (raw) cargado.")
except Exception as e:
    print(f"Error al cargar: {e}")
    df_titanic_raw = pd.DataFrame() # DataFrame vacío para evitar errores posteriores

if not df_titanic_raw.empty:
    display(df_titanic_raw.head())
    print("Información inicial del dataset raw:")
    df_titanic_raw.info()
    
    # Separaremos características (X) y variable objetivo (y) para simular un escenario de ML
    # 'survived' es nuestra variable objetivo. El resto (o un subconjunto) son características.
    if 'survived' in df_titanic_raw.columns:
        X = df_titanic_raw.drop('survived', axis=1)
        y = df_titanic_raw['survived']
        print(f"\nForma de X: {X.shape}, Forma de y: {y.shape}")
    else:
        print("Advertencia: La columna 'survived' no se encuentra. Se usará todo el DataFrame como X.")
        X = df_titanic_raw.copy()
        y = None
else:
    X = pd.DataFrame() # DataFrame vacío
    y = pd.Series(dtype='float64') if pd.Series is not None else None

Dataset del Titanic (raw) cargado.


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Información inicial del dataset raw:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB

Forma de X: (891, 14), Forma de y: (891,)


**Nota Importante sobre `fit` y `transform`:**
En un flujo de trabajo de Machine Learning real:
1.  Dividirías `X` e `y` en `X_train`, `X_test`, `y_train`, `y_test`.
2.  Ajustarías (`fit`) el pipeline de preprocesamiento **únicamente** en `X_train`.
3.  Aplicarías la transformación (`transform`) tanto a `X_train` como a `X_test`.
Para simplificar esta demostración del pipeline en sí, podríamos ajustar el preprocesador en todo `X`, pero ten en mente que esto no es lo que harías antes de entrenar un modelo.

---

## 3. Definición de Transformadores y Pipelines por Tipo de Columna

Primero, identificamos qué columnas son numéricas y cuáles son categóricas. Luego, definimos las transformaciones que queremos aplicar a cada grupo.

In [4]:
if not X.empty:
    # Identificar columnas numéricas y categóricas
    # (Excluimos columnas que podríamos querer dropear directamente o tratar de forma especial como 'name', 'ticket', 'cabin')
    cols_numericas = ['age', 'fare', 'sibsp', 'parch', 'pclass'] # pclass es numérica pero representa categorías
    cols_categoricas_onehot = ['sex', 'embarked'] # 'deck', 'embark_town', 'who', 'adult_male' también son categóricas
    
    # Asegurarnos que las columnas existan en X
    cols_numericas_existentes = [col for col in cols_numericas if col in X.columns]
    cols_categoricas_onehot_existentes = [col for col in cols_categoricas_onehot if col in X.columns]
    
    print(f"Columnas numéricas a procesar: {cols_numericas_existentes}")
    print(f"Columnas categóricas (para OneHot) a procesar: {cols_categoricas_onehot_existentes}")
    
    # Pipeline para transformaciones numéricas
    pipeline_numerico = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')), # Imputar NaNs con la mediana
        ('scaler', StandardScaler())                   # Escalar los datos
    ])
    
    # Pipeline para transformaciones categóricas (One-Hot Encoding)
    pipeline_categorico_onehot = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')), # Imputar NaNs con la moda
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # Convertir a dummies
    ])
    
    # (Opcional) Podríamos tener un pipeline para 'pclass' si quisiéramos tratarla como categórica
    # pipeline_pclass = Pipeline(steps=[
    #     ('imputer', SimpleImputer(strategy='most_frequent')),
    #     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    # ])
    # cols_pclass = ['pclass']
    # Y luego quitar 'pclass' de cols_numericas_existentes.
    # Por simplicidad, aquí la dejamos como numérica y la escalaremos.

else:
    print("El DataFrame X está vacío. No se pueden definir pipelines.")
    pipeline_numerico = None
    pipeline_categorico_onehot = None
    cols_numericas_existentes = []
    cols_categoricas_onehot_existentes = []

Columnas numéricas a procesar: ['age', 'fare', 'sibsp', 'parch', 'pclass']
Columnas categóricas (para OneHot) a procesar: ['sex', 'embarked']


---

## 4. Construcción del `ColumnTransformer`

El `ColumnTransformer` nos permite aplicar diferentes pipelines (o transformadores individuales) a diferentes subconjuntos de columnas del DataFrame.

In [5]:
if not X.empty and pipeline_numerico and pipeline_categorico_onehot:
    # Crear el ColumnTransformer
    # Lista de tuplas: (nombre_transformador, pipeline_o_transformador, columnas_a_aplicar)
    preprocesador = ColumnTransformer(
        transformers=[
            ('num', pipeline_numerico, cols_numericas_existentes),
            ('cat_onehot', pipeline_categorico_onehot, cols_categoricas_onehot_existentes)
            # ('cat_pclass', pipeline_pclass, cols_pclass) # Si hubiéramos definido pipeline_pclass
        ],
        remainder='drop' # 'drop' las columnas no especificadas. 'passthrough' las mantendría sin cambios.
    )
    print("ColumnTransformer creado.")
else:
    print("No se pudo crear ColumnTransformer porque X o los pipelines no están definidos.")
    preprocesador = None

ColumnTransformer creado.


---

## 5. Aplicación del Pipeline de Preprocesamiento al Dataset

Ahora, aplicamos el `preprocesador` (que es un `ColumnTransformer`) a nuestros datos `X`.
El método `fit_transform()` ajustará los imputers, scalers y encoders a los datos y luego los transformará.

In [6]:
if preprocesador and not X.empty:
    try:
        X_preprocesado_array = preprocesador.fit_transform(X)
        print("\nDataset X preprocesado exitosamente.")
        print(f"Forma del array X_preprocesado: {X_preprocesado_array.shape}")
        
        # Mostrar algunas filas del array resultante
        print("\nPrimeras 5 filas del array X_preprocesado:")
        print(X_preprocesado_array[:5])
        
        # (Opcional) Convertir de nuevo a DataFrame para inspección
        # Obtener los nombres de las características después de la transformación
        # Es un poco más complejo obtener los nombres de características de OneHotEncoder dentro de ColumnTransformer
        try:
            nombres_features_numericas = cols_numericas_existentes
            # Para OneHotEncoder, si está en el pipeline 'cat_onehot' y es el segundo paso ('onehot')
            nombres_features_categoricas_onehot = preprocesador.named_transformers_['cat_onehot'].named_steps['onehot'].get_feature_names_out(cols_categoricas_onehot_existentes)
            
            nombres_features_finales = nombres_features_numericas + list(nombres_features_categoricas_onehot)
            
            X_preprocesado_df = pd.DataFrame(X_preprocesado_array, columns=nombres_features_finales, index=X.index)
            print("\n--- DataFrame X Preprocesado (Primeras Filas) ---")
            display(X_preprocesado_df.head())
        except Exception as e_feature_names:
            print(f"Error al obtener nombres de características o crear DataFrame: {e_feature_names}")
            X_preprocesado_df = pd.DataFrame(X_preprocesado_array) # Sin nombres de columna
            print("\n--- DataFrame X Preprocesado (Primeras Filas, sin nombres de columna) ---")
            display(X_preprocesado_df.head())
            
    except Exception as e_transform:
        print(f"Error durante fit_transform del preprocesador: {e_transform}")
        X_preprocesado_array = None
        X_preprocesado_df = None
else:
    print("Preprocesador o X no están definidos. No se puede aplicar.")
    X_preprocesado_array = None
    X_preprocesado_df = None


Dataset X preprocesado exitosamente.
Forma del array X_preprocesado: (891, 10)

Primeras 5 filas del array X_preprocesado:
[[-0.56573646 -0.50244517  0.43279337 -0.47367361  0.82737724  0.
   1.          0.          0.          1.        ]
 [ 0.66386103  0.78684529  0.43279337 -0.47367361 -1.56610693  1.
   0.          1.          0.          0.        ]
 [-0.25833709 -0.48885426 -0.4745452  -0.47367361  0.82737724  1.
   0.          0.          0.          1.        ]
 [ 0.4333115   0.42073024  0.43279337 -0.47367361 -1.56610693  1.
   0.          0.          0.          1.        ]
 [ 0.4333115  -0.48633742 -0.4745452  -0.47367361  0.82737724  0.
   1.          0.          0.          1.        ]]

--- DataFrame X Preprocesado (Primeras Filas) ---


,age,fare,sibsp,parch,pclass,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,-0.565736,-0.502445,0.432793,-0.473674,0.827377,0.0,1.0,0.0,0.0,1.0
1,0.663861,0.786845,0.432793,-0.473674,-1.566107,1.0,0.0,1.0,0.0,0.0
2,-0.258337,-0.488854,-0.474545,-0.473674,0.827377,1.0,0.0,0.0,0.0,1.0
3,0.433312,0.420730,0.432793,-0.473674,-1.566107,1.0,0.0,0.0,0.0,1.0
4,0.433312,-0.486337,-0.474545,-0.473674,0.827377,0.0,1.0,0.0,0.0,1.0


**Comentario sobre el Pipeline Completo:**
Podríamos haber envuelto nuestro `ColumnTransformer` en un `Pipeline` más grande, especialmente si quisiéramos añadir un modelo al final:
```python
# from sklearn.linear_model import LogisticRegression
# pipeline_completo = Pipeline(steps=[
#     ('preprocesador', preprocesador),
#     ('clasificador', LogisticRegression()) 
# ])
# pipeline_completo.fit(X_train, y_train)
# predicciones = pipeline_completo.predict(X_test)
```
Esto ilustra cómo el preprocesamiento se integra limpiamente en el flujo de trabajo de modelado.

In [8]:
display(X_preprocesado_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         891 non-null    float64
 1   fare        891 non-null    float64
 2   sibsp       891 non-null    float64
 3   parch       891 non-null    float64
 4   pclass      891 non-null    float64
 5   sex_female  891 non-null    float64
 6   sex_male    891 non-null    float64
 7   embarked_C  891 non-null    float64
 8   embarked_Q  891 non-null    float64
 9   embarked_S  891 non-null    float64
dtypes: float64(10)
memory usage: 69.7 KB


None

---

## 6. Resumen

Los `Pipeline` y `ColumnTransformer` de Scikit-learn son herramientas increíblemente útiles para:
* Organizar y simplificar tu código de preprocesamiento.
* Aplicar diferentes transformaciones a diferentes tipos de columnas de manera sistemática.
* Asegurar consistencia entre los datos de entrenamiento y prueba, previniendo la fuga de datos.
* Facilitar la experimentación con diferentes pasos de preprocesamiento y modelos.

Dominar estas herramientas te permitirá construir flujos de trabajo de Machine Learning mucho más robustos y profesionales.

---

## 7. Ejercicios Prácticos 🧩

Utiliza el dataset de empleados (crudo, con nulos y diferentes tipos de datos) para construir un pipeline de preprocesamiento.

### Ejercicio 7.1: Pipeline de Preprocesamiento para Datos de Empleados

**Dataset (versión cruda):**
```python
datos_empleados_crudo = {
    'ID_Empleado': ['E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E02', 'E07', 'E08', 'E09', 'E10', 'E11'],
    'Nombre': ['Carlos', 'Ana', 'Luis', 'Sofia', 'Pedro', 'Laura', 'Ana', 'David', 'Maria', 'Juan', np.nan, 'Elena'],
    'Edad': [34, 28, 45, 30, np.nan, 25, 28, 50, 33, 200, 40, 29], # Nan, Outlier
    'Departamento': ['Ventas', 'Marketing', 'TI', 'Ventas', 'TI', np.nan, 'Marketing', 'RRHH', 'Ventas', 'TI', 'Marketing', 'Ventas'], # Nan
    'SalarioAnual': [60000.0, 75000.0, 90000.0, 62000.0, 85000.0, np.nan, 75000.0, 110000.0, 58000.0, 500000.0, 72000.0, np.nan], # Nan, Outlier
    'AniosExperiencia': [5, 3, 10, 4, 7, 2, 3, 15, 6, 1, 8, np.nan] # Nan
}
df_empleados_crudo = pd.DataFrame(datos_empleados_crudo)
```

**Tareas:**
1.  **Crea el DataFrame** `df_empleados_crudo`.
2.  **Define las columnas:** Identifica las columnas numéricas (`Edad`, `SalarioAnual`, `AniosExperiencia`) y categóricas (`Departamento`). Las columnas `ID_Empleado` y `Nombre` pueden ser ignoradas o eliminadas por el `ColumnTransformer` (usando `remainder='drop'`).
3.  **Crea los pipelines individuales:**
    * `pipeline_numerico_emp`: Debe usar `SimpleImputer` (estrategia 'median') y `StandardScaler`.
    * `pipeline_categorico_emp`: Debe usar `SimpleImputer` (estrategia 'most_frequent') y `OneHotEncoder` (`handle_unknown='ignore'`, `sparse_output=False`).
4.  **Construye el `ColumnTransformer`** (`preprocesador_empleados`) para aplicar estos pipelines a las columnas correspondientes.
5.  **Aplica el `preprocesador_empleados`** al DataFrame `df_empleados_crudo` (puedes excluir `ID_Empleado` y `Nombre` antes de pasarlo al `fit_transform` o dejar que `remainder='drop'` lo haga).
6.  **Resultado:**
    * Muestra la forma (shape) del array resultante.
    * Muestra las primeras 5 filas del array resultante.
    * (Opcional Avanzado) Intenta convertir el array resultante de nuevo a un DataFrame con nombres de columna apropiados.

In [ ]:
# 1. Crear DataFrame
datos_empleados_crudo = {
    'ID_Empleado': ['E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E02', 'E07', 'E08', 'E09', 'E10', 'E11'],
    'Nombre': ['Carlos', 'Ana', 'Luis', 'Sofia', 'Pedro', 'Laura', 'Ana', 'David', 'Maria', 'Juan', np.nan, 'Elena'],
    'Edad': [34, 28, 45, 30, np.nan, 25, 28, 50, 33, 200, 40, 29],
    'Departamento': ['Ventas', 'Marketing', 'TI', 'Ventas', 'TI', np.nan, 'Marketing', 'RRHH', 'Ventas', 'TI', 'Marketing', 'Ventas'],
    'SalarioAnual': [60000.0, 75000.0, 90000.0, 62000.0, 85000.0, np.nan, 75000.0, 110000.0, 58000.0, 500000.0, 72000.0, np.nan],
    'AniosExperiencia': [5, 3, 10, 4, 7, 2, 3, 15, 6, 1, 8, np.nan]
}
df_empleados_crudo = pd.DataFrame(datos_empleados_crudo)
print("--- DataFrame Original de Empleados (Ejercicio) ---")
display(df_empleados_crudo.head())
df_empleados_crudo.info()

# 2. Definir columnas
cols_num_emp = ['Edad', 'SalarioAnual', 'AniosExperiencia']
cols_cat_emp = ['Departamento']

# 3. Crear pipelines individuales
pipeline_numerico_emp = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

pipeline_categorico_emp = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 4. Construir ColumnTransformer
preprocesador_empleados = ColumnTransformer(
    transformers=[
        ('num', pipeline_numerico_emp, cols_num_emp),
        ('cat', pipeline_categorico_emp, cols_cat_emp)
    ],
    remainder='drop' # Ignorar 'ID_Empleado' y 'Nombre'
)

# 5. Aplicar el preprocesador
try:
    X_empleados_procesado_array = preprocesador_empleados.fit_transform(df_empleados_crudo)
    
    # 6. Resultados
    print(f"\nForma del array de empleados procesado: {X_empleados_procesado_array.shape}")
    print("\nPrimeras 5 filas del array de empleados procesado:")
    print(X_empleados_procesado_array[:5])

    # Opcional Avanzado: Convertir a DataFrame con nombres de columna
    try:
        nombres_num_emp = cols_num_emp
        nombres_cat_emp_ohe = preprocesador_empleados.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cols_cat_emp)
        nombres_features_empleados = nombres_num_emp + list(nombres_cat_emp_ohe)
        
        df_empleados_procesado = pd.DataFrame(X_empleados_procesado_array, columns=nombres_features_empleados)
        print("\n--- DataFrame de Empleados Procesado (Primeras Filas) ---")
        display(df_empleados_procesado.head())
        df_empleados_procesado.info()
    except Exception as e_feat_names_emp:
        print(f"Error al obtener nombres de características para empleados: {e_feat_names_emp}")
        # Mostrar como array si falla la reconstrucción del DataFrame
        df_empleados_procesado = pd.DataFrame(X_empleados_procesado_array)
        display(df_empleados_procesado.head())
        
except Exception as e_pipeline_emp:
    print(f"Error al aplicar el pipeline de empleados: {e_pipeline_emp}")